## **1.0 Library & File Preparation**

In [ ]:
library(MASS)
library(tidyverse)
suppressMessages(library(caret)) 
list.files(path = "../input")

In [ ]:
# Make sure the data is available
if (length(list.files("../input", pattern = "recognition")) > 0) {
    
    # Copy all files to the current directory
    system("cp -r ../input/bda-2021-physical-activity-recognition/* ./")
    
} else {
    
    # Download data for this competition
    data_url = "https://phonesensordata.netlify.app/Archive.zip"
    download.file(data_url, "Archive.zip")

    # Unzip all files in the current directory
    unzip("Archive.zip")
    
}

# list files in the current working directory
list.files()

# show the content of the labels file 
file.show("activity_labels.txt")

The signals themselves are stored in text files. In these files there are three columns; each column is the signal measured in one of the 3 channels of the sensor (these channels are associated in X, Y and Z direction). Each signal consists of a sequence of measurements, called _samples_.


## **2.0 Label Preperation**
Make each sample labeled with an activity.

In [ ]:
# Import label
act_labels = read_delim("activity_labels.txt"," ",col_names=F,trim_ws=T) 
act_labels = act_labels %>% select(X1,X2)

labels = read_delim("./RawData/Train/labels_train.txt", " ", col_names = F)
colnames(labels) <- c('trial', 'userid', 'activity', 'start', 'end')

labels = labels %>% mutate(activity = act_labels$X2[activity])

# Add the sequence start:end to each row in a list.
sample_labels_nested = 
    labels %>% 
    rowwise() %>% # do next operation(s) rowwise
    mutate(sampleid = list(start:end)) %>%
    ungroup()


# Unnest the nested tabel.
sample_labels = 
    sample_labels_nested %>% 

    # Rows are segments, we need to keep track of different segements
    mutate(segment = row_number() ) %>% 

    # Expand the data frame to one sample per row
    unnest() %>% 

    # Remove columns we don't need anymore
    select(-start, -end) 


# Check the result (first few rows are not interesting; rows 977-990 are)
print(sample_labels[977:990, ])

# 3.0 **Feature Extraction from Signals**

Several features can be extracted from signals. We extracted the following:

### Statistical features:
* **Power:** represents the average squared amplitude of the signal.
* **Energy:** the “total power” of the n samples.
* **Entropy:** a measure of average surprise in handling signals: ‘Surprise’ about an observed value can be expressed in terms of the negative logarithm of the probability of that outcome; the higher the probability, the less ‘surprised’ you are.
* **Mean**
* **Median**
* **Min and max**
* **Mean absolute deviation**
* **Correlation**

### Time domain features:
* **Cosine angle:** the scaled inner product. This time domain feature takes the ordering of the sequence of numbers into account.
* **Lagged correlation**

### Frequency domain features:
* **Mean frequency:** the sum of the product of the spectrogram intensity (in dB) and the frequency, divided by the total sum of spectrogram intensity.
* **Standard deviation:** a measure for how much the frequencies in a spectrum can deviate from the centre of gravity.
* **Spectrum peak**
* **Skewness:** a measure for how much the shape of the spectrum below the centre of gravity is different from the shape above the mean frequency.
* **Kurtosis:** provides a measure of the “peakedness” of a random signal.

### Feature choice explanation

We used the data from all measurements for our predictors (so X1, X2 and X3). We chose all our predictors because they seemed to vary strongly across the different activities. 

* From the histograms plotted it follows that both the mean and standard deviation from X1, X2 and X3 are very different for a lot of activities. This is for example clear when looking at X1 in the histograms for ‘laying’ and ‘walking upstairs’. In ‘walking upstairs’ the mean of X1 is evidently a lot higher. 
* Another predictor we chose is the autocorrelation of the subsequent measurements of the same variable (lagged correlation). A high autocorrelation occurs when a predictor doesn’t differ a lot among measurements as is for example visible in the histogram for ‘standing’. In this histogram neither of the measurements X1, X2 and X3 seem to vary a lot, contrary to a lot of the other histograms. 
* We also included skewness (how symmetric the distribution of the histogram is) and kurtosis (how much the histogram is ‘spread out) as predictors. These statistics also vary greatly across the different activities. The distribution of X1 is for example quite symmetrical for ‘walking upstairs’ and is very flat (low kurtosis) for ‘lie to stand’, so these measures should be suitable to distinguish among the different activities. 
* Lastly we included entropy as a predictor for the activity because it seems that for some activities the measurements seem to follow a certain pattern and are a bit more predictable, while in other they are more chaotic.

### Feature functions

In [ ]:
# Helper functions
most_common_value = function(x) {
    counts = table(x, useNA = 'no')
    most_frequent = which.max(counts)
    return(names(most_frequent))
}

## Feature functions
lagged_cor = function(x, y = x, lag = 0) {
    # compute correlation between x and a time shifted y
    r_lagged = cor(x, dplyr::lag(y, lag), use = 'pairwise')
    return(r_lagged)
}


# entropy
entropy  <- function(x, nbreaks = nclass.Sturges(x)) {
    r = range(x)
    x_binned = findInterval(x, seq(r[1], r[2], len = nbreaks))
    h = tabulate(x_binned, nbins = nbreaks) # fast histogram
    p = h/sum(h)
    -sum(p[p>0] * log(p[p>0]))
}


# spectrum_peak (code from group 10 and 3)
spectrum_peak <- function(x){
    spec = spectrum(x, log = 'n', plot = FALSE)$spec
    entropy = entropy(spec)
    return(entropy)
}


#spectrum_mean (code from group 10 and 3)
spectrum_mean <- function(x) {
    
    spec = spectrum(x, log = 'n', plot = FALSE)$spec
    freq = spectrum(x, log = 'n', plot = FALSE)$freq
    df = freq[2] - freq[1]
    
    spec_mean <- sum(freq * spec * df)
    
    if(is.na(spec_mean)) {
        return(0)
    } else {
        return(spec_mean)
    }
}

#spectrum_sd (code from group 10 and 3)
spectrum_sd <- function(x) {
    spec = spectrum(x, log = 'n', plot = FALSE)$spec
    freq = spectrum(x, log = 'n', plot = FALSE)$freq
    df = freq[2] - freq[1]
    
    return(sqrt(sum((freq - mean(x))^2 * spec * df)))
}


### Function of putting things all together


In [ ]:
extractTimeDomainFeatures <- function(filename, sample_labels) {
    
    # extract user and experimental run ID's from file name
    username = gsub(".+user(\\d+).+", "\\1", filename) %>% as.numeric()
    expname  = gsub( ".+exp(\\d+).+", "\\1", filename) %>% as.numeric()
    
    # import the sensor signals from the file
    user01 <- read_delim(filename, " ", col_names = F, progress = TRUE, 
                 col_types = "ddd")
    
    
    # merge signals with labels 
    user_df <- 
        data.frame(userid = username, trial = expname, user01) %>%
        mutate(sampleid = 0:(nrow(user01)-1) ) %>%
        left_join(sample_labels, by = c('userid','trial','sampleid')) 

    
    # split in epochs of 128 samples and compute features per epoch
    usertimedom <-  user_df %>%
    
          # add an epoch ID variable (on epoch = 2.56 sec)
          mutate(epoch = sampleid %/% 128) %>% 

          # extract statistical features from each epoch
          group_by(epoch) %>%
          summarise(
            # keep track of user and experiment information
            user_id = username, 
            exp_id = expname,   
              
            # epoch's activity labels and start sample
            activity = most_common_value(c("-", activity)),
            sampleid = sampleid[1],
              
            # features
            
            # mean
            m1 = mean(X1), 
            m2 = mean(X2),
            m3 = mean(X3),
            
            # median
            median1 = median(X1),
            median2 = median(X2),
            median3 = median(X3),
            
            # min and max
            min1 = min(X1),
            min2 = min(X2),
            min3 = min(X3),
            max1 = max(X1),
            max3 = max(X2),
            max3 = max(X3),
              
            # mean absolute deviation (from group 10)
            mad1 = mad(X1), 
            mad2 = mad(X2),   
            mad3 = mad(X3),
           
            # variation
            var1 = var(X1),
            var2 = var(X2),
            var3 = var(X3),
              
            # standard deviation
            sd1 = sd(X1), 
            sd2 = sd(X2), 
            sd3 = sd(X3),              

            
            # lagged correlation
            AR1_1 = lagged_cor(X1, lag = 1),
            AR2_1 = lagged_cor(X1, lag = 1),
            AR3_1 = lagged_cor(X1, lag = 1),
            AR12_1 = lagged_cor(X1, X2, lag = 1),
            AR13_1 = lagged_cor(X1, X3, lag = 1),
            AR23_1 = lagged_cor(X2, X3, lag = 1),

              
            # correlation  
            cor1_2 = cor(X1, X2),
            cor1_3 = cor(X1, X3), 
            cor2_3 = cor(X2, X3),
              
            
            # entropy
            entropy1 = entropy(X1),
            entropy2 = entropy(X2),
            entropy3 = entropy(X3),
              
            # power
            power1 = mean(X1^2),
            power2 = mean(X2^2),
            power3 = mean(X3^2),
            
            # quartiles
            q1_25 = quantile(X1, .25),
            q2_25 = quantile(X2, .25),
            q3_25 = quantile(X3, .25),
            q1_75 = quantile(X1, .75),
            q2_75 = quantile(X2, .75),
            q3_75 = quantile(X3, .75),
              
            # skewness
            skew1 = e1071::skewness(X1),
            skew2 = e1071::skewness(X2),
            skew3 = e1071::skewness(X3), 
            
            # kurtosis
            kurt1 = e1071::kurtosis(X1),
            kurt2 = e1071::kurtosis(X2),
            kurt3 = e1071::kurtosis(X3),
              
            # cosine angle (code from group 10)
            cosangle1 = X1 %*% X2 / sqrt(sum(X1^2) * sum(X2^2)), 
            cosangle2 = X1 %*% X3 / sqrt(sum(X1^2) * sum(X3^2)),  
            cosangle3 = X3 %*% X2 / sqrt(sum(X3^2) * sum(X2^2)),
            n_samples = n(),
             
            # spectrum features (code from group 10 and 3)
            spectrum1 = spectrum_mean(X1), 
            spectrum2 = spectrum_mean(X2),
            spectrum3 = spectrum_mean(X3),
            spectrumsd1 = spectrum_sd(X1),   
            spectrumsd2 = spectrum_sd(X2),
            spectrumsd3 = spectrum_sd(X3),
            spectrumpeak1 = spectrum_peak(X1),
            spectrumpeak2 = spectrum_peak(X2),
            spectrumpeak3 = spectrum_peak(X3) 
          ) 
    
    usertimedom 
}

# 4.0 **Import Training Data & Cleaning**

## Import

In [ ]:
# run this for all files of both the 'acc' and 'gyro' data
filenames <- dir("./RawData/Train/", "^acc", full.names = TRUE) 
filenames_gyro <- dir("./RawData/Train/", "^gyro", full.names = TRUE) 


myData = map_dfr(filenames, extractTimeDomainFeatures, sample_labels) 
myData2 = map_dfr(filenames_gyro, extractTimeDomainFeatures, sample_labels)


myData <- myData %>%
left_join(myData2, by = c("user_id", "epoch", "exp_id", "activity", "sampleid", "n_samples"), 
          suffix = c(".acc", ".gyro"))


head(myData) 
glimpse(myData)

## Cleaning

- Remove unlabeled epochs
- keep epochs defined in the sample submission file
- Remove epochs that do not consist of 128 samples
- Delete unuseful columns: epoch,user_id, exp_id, sampleid, n_samples
- Finding non zero variation
- Remove NA's
- Remove Highly correlated variables



## Disclaimer

We only found out very last minute that the way in which we cleaned the data (see code below) was probably not right. Because we removed certain rows with no lables the order of our predictions did not match the order of the test set. Hence we think our score on the leaderboard is quite low, while the accuracy of our best model is almost 0.9. However trying to fix the data gave some new errors that we did not have time to fix last minute. 

In [ ]:
## Data cleaning
# Remove unlabeled epochs
myData_labeled <- subset(myData, epoch != "-")

# Only keep epochs defined in the sample submission file
myData_labeled <- subset(myData, activity != "-")

# Remove epochs that do not consist of 128 samples
myData_sampeled <- subset(myData_labeled, n_samples = 128)

# Delete unuseful columns: epoch,user_id, exp_id, sampleid, n_samples
myData_useful <- subset(myData_sampeled, select = -c(epoch,user_id, exp_id, sampleid, n_samples))

# Finding non zero variation
near_zero <- caret::nearZeroVar(myData_useful) # no non zero variation

# Remove NA's
any(is.na(myData_useful)) # no NA's

# Highly correlated variables,remove them
high_cor_var <- caret::findCorrelation(cor(myData_useful[,-c(1)])) # remove activity column to make it numeric
high_cor_var_correct <- high_cor_var + 1

myData_clean <- myData_useful[,-high_cor_var_correct]

head(myData_clean)

# **5.0 Model Fitting**
- Logistic regression
- LDA
- QDA
- k-NN

In [ ]:
# cross validation
trcntr = trainControl('cv', number = 10, p=0.8)

## Logistic Regression

In [ ]:
#fit_multinom = caret::train(activity ~ ., data = myData_clean, method="multinom", trControl = trainControl('cv', number = 2, p=0.8))

#fit_multinom

## LDA: Linear Discriminant Analysis

In [ ]:
## LDA Model
fit_lda = caret::train(activity ~ ., data = myData_clean, method="lda", trControl = trcntr)

fit_lda

## QDA: Quadratic discriminant analysis

We ran this model but it took a very long time and had poor accuracy so we decided to comment it out.

In [ ]:
## QDA Model 

#fit_qda = caret::train(activity ~ ., data = myData_clean, method="qda", trControl = trcntr)

#fit_qda

## KNN: k-nearest neighbors

In [ ]:
## k-Nearest Neighbours Model
fit_knn = caret::train(activity ~ ., data = myData_clean, method="knn", trControl = trcntr)

fit_knn

In the end we used the scaled k-Nearest Neighbours Model for our predictions, because it has the highest accuracy as becomes clear from the barplot below. We also did include highly correltated predictors in the model, because this improved the accuracy even more, eventhough is makes the model more complex.  

In [ ]:
## Scaled k-Nearest Neighbours Model
fit_knns = caret::train(activity ~ .,
               data = myData_clean, method="knn", trControl = trcntr, preProcess = "scale")
fit_knns_all = caret::train(activity ~ .,
               data = myData_useful, method="knn", trControl = trcntr, preProcess = "scale")


# **6.0 Comparing Model Performances**

# Performance visualization

A simple and effective visualization of the performance accuracy is a `barplot`. 

In [ ]:
## Visualize model performance differences


models = list(lda = fit_lda, knn = fit_knn, knns = fit_knns, knns_full = fit_knns_all )

# extract the cross-validated accuracies from each model
Acc = sapply(models, function(mdl) max(mdl$results$Accuracy)) 

# make a barplot with only the best performing model in red
color = 1 + (Acc >= max(Acc)) 
barplot(Acc, horiz=T, las=1, col = color)

# **7.0 Prediction**

In [ ]:
# Import test data

filenames_test_acc <- dir("./RawData/Test/", "^acc", full.names = TRUE)
filenames_test_gyro <- dir("./RawData/Test/", "^gyro", full.names = TRUE) 

test_data_acc = map_dfr(filenames_test_acc, extractTimeDomainFeatures, sample_labels) 
test_data_gyro = map_dfr(filenames_test_gyro, extractTimeDomainFeatures, sample_labels)


test_data <- test_data_acc %>%
left_join(test_data_gyro, by = c("user_id", "epoch", "exp_id", "activity", "sampleid", "n_samples"), 
          suffix = c(".acc", ".gyro"))

glimpse(test_data)

# Predict
test_result <- predict(fit_knns_all, test_data)

test_data['activity'] <- test_result

# **8.0 Submissions**

In [ ]:
test_data %>%

    # prepend "user" and "exp" to user_id and exp_id
    mutate(
        user_id = paste(ifelse(user_id < 10, "user0", "user"), user_id, sep = ""), 
        exp_id = paste(ifelse(exp_id < 10, "exp0", "exp"), exp_id, sep = "")
    ) %>% 

    # unit columnes user_id, exp_id and sample_id into a string 
    # separated by "_" and store it in the new variable `Id`
    unite(Id, user_id, exp_id, sampleid) %>%

    # retain only the `Id` and  predictions
    dplyr::select(Id, Predicted = activity) %>%

    # write to file
    write_csv("test_set_predictions.csv")


# Check the result: print first 20 lines in the submission file
cat(readLines("test_set_predictions.csv",20), sep = "\n")

# Division of Labor:
**Britt:**
* Writing code for data cleaning
* Writing code for model fitting
* Code styling
* Finding features and implementing

**Zena:**
* Model fitting, prediction
* Find useful features
* Tidy document, debug

**Jesse:**
* Adding the gyro data
* Adding basic predictors and spectrum predictors
* Explaining our choice of features
* Explaining model selection